In [29]:
from ansatz_simulation_class import AnsatzSimulation
import torch
from genetic_quantum import QuantumModel, QuanvLayer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from patch_making import PatchExtraction, Quanv_2d
from torch import tensor
import matplotlib.pyplot as plt
from sampled_cv_dataset import SampledDataset4Training
import time

### Defining Classical Module class

In [73]:
class L2NormalizationLayer(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2NormalizationLayer, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

class HybridModel(nn.Module):
    def __init__(self, n_qubits, patch_size, chromosome, num_classes, input_size, mode='2d',):
        super().__init__()
        self.quanv_layer = QuanvLayer(
            n_qubits=n_qubits,
            patch_size=patch_size,
            chromosome=chromosome,
            mode=mode
        )
        feature_size = input_size
        self.fc1 = nn.Linear(feature_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.norm = nn.LayerNorm(feature_size)
        self.fc = nn.Linear(feature_size, num_classes)
        self.l2norm = L2NormalizationLayer(dim=1)
        
    def forward(self, x):
        print("Passing through quanvolution layer...")
        start_time = time.perf_counter()
        x = self.quanv_layer.forward(x)
        end_time =   time.perf_counter()
        print(f'Quanvolution processing time: {end_time - start_time}')  
        x = x.flatten(start_dim=1)
        #print(x.shape)
        #x = self.l2norm(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

### Instantiating genetic ansatz model

In [76]:
n_qubits = 4
max_patches = 50
patch_size = 2
batch_size = 100
num_classes = 2
input_size = max_patches * n_qubits
num_epochs = 4
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['ry_gate', 'rz_gate', 'ry_gate', 'rx_gate'],['trgt_0', 'ctrl_0', 'trgt_1', 'ctrl_1'], ['rz_gate', 'trgt_0', 'ctrl_0', 'ry_gate']]
#simple_fc = ClassicalComponent(num_classes, input_size)
hqcnn = HybridModel(n_qubits, patch_size, toy_chromosome, num_classes, input_size)
hqcnn

HybridModel(
  (quanv_layer): QuanvLayer()
  (fc1): Linear(in_features=200, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (norm): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=200, out_features=2, bias=True)
  (l2norm): L2NormalizationLayer()
)

### Getting data and sampling it

In [32]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size, max_patches)])
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


In [33]:
len(mnist_dataset)

60000

In [34]:
selected_classes = [0, 1]
reduced_mnist = [mnist_dataset.__getitem__(index) for index in range(mnist_dataset.__len__()) if mnist_dataset.__getitem__(index)[1] in selected_classes]
len(reduced_mnist)

12665

In [35]:
mnist_dataset.__getitem__(4)

(tensor([[0.0000, 0.9059, 0.9922, 0.9882],
         [0.9882, 0.8902, 0.9059, 0.0000],
         [0.7725, 0.9922, 0.9843, 0.0000],
         [0.9922, 0.5412, 0.0941, 0.9882],
         [0.9216, 0.8510, 0.1647, 0.7529],
         [0.8235, 0.9882, 0.6588, 0.0000],
         [0.9451, 0.9882, 0.3020, 0.0000],
         [0.9882, 0.9922, 0.0000, 0.0000],
         [0.0000, 0.0627, 0.9882, 0.9882],
         [0.5804, 0.9922, 0.4431, 0.5804],
         [0.2157, 0.8235, 0.9922, 0.3412],
         [0.0000, 0.0000, 0.9098, 0.9922],
         [0.8863, 0.9882, 0.0000, 0.0000],
         [0.0549, 0.0000, 0.9882, 0.8824],
         [0.0549, 0.0000, 0.8431, 0.9882],
         [0.0000, 0.3412, 0.9882, 0.7412],
         [0.7176, 0.0000, 0.3608, 0.9882],
         [0.9882, 0.0000, 0.0000, 0.7882],
         [0.7137, 0.9922, 0.0000, 0.0000],
         [0.6902, 0.0000, 0.1412, 0.9882],
         [0.9922, 0.6902, 0.0000, 0.0314],
         [0.0157, 0.9490, 0.7451, 0.0196],
         [0.0000, 0.0000, 0.9922, 0.5725],
         [0

In [36]:
class_one = 0
class_two = 0
binary_dataset = []

for image in reduced_mnist:
    if image[1] == 0 and class_one < 200:
        binary_dataset.append(image)
        class_one +=1 
    elif image[1] == 1 and class_two < 200:
        binary_dataset.append(image)
        class_two += 1
        
    if class_one == 200 and class_two == 200:
        break

In [37]:
images, labels = zip(*binary_dataset)
mnist_images = torch.stack(images)

In [38]:
mnist_images.shape

torch.Size([400, 50, 4])

In [39]:
mean = torch.mean(mnist_images)
mean

tensor(0.4163)

In [40]:
diffs = mnist_images - mean
var = torch.mean(torch.pow(diffs, 2.0))
var

tensor(0.1968)

In [41]:
std = torch.pow(var, 0.5)
std

tensor(0.4436)

### Training my model

In [42]:
#quantum_processing = transforms.Compose([Quanv_2d(n_qubits, toy_chromosome)])
training_zeros_ones = SampledDataset4Training(binary_dataset)
training_loader = DataLoader(training_zeros_ones, batch_size, shuffle=True)

In [60]:
optimizer = torch.optim.Adam(hqcnn.parameters(), lr=0.1, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()
#loss = loss_fn(output, mnist_labels)
device = torch.device("cpu")

In [44]:
mean = 0.
meansq = 0.
for (data, label) in training_loader:
    mean = data.mean()
    meansq = (data**2).mean()

std = torch.sqrt(meansq - mean**2)
print("mean: " + str(mean))
print("std: " + str(std))

mean: tensor(0.4046)
std: tensor(0.4424)


In [45]:
post_processing = [data for (data, label) in training_loader]

In [46]:
post_process = torch.stack(post_processing).squeeze(0)
post_process.shape

torch.Size([4, 100, 50, 4])

In [47]:
mean_p = torch.mean(post_process)
mean_p

tensor(0.4163)

In [48]:
p_diffs = post_process - mean_p
p_var = torch.mean(torch.pow(p_diffs, 2.0))
p_var

tensor(0.1968)

In [49]:
p_std = torch.pow(p_var, 0.5)
p_std

tensor(0.4436)

In [50]:
phi_n = F.normalize(post_process.flatten(start_dim=1), dim=1)
cos_sim = phi_n @ phi_n.T
cos_sim.mean()

tensor(0.6441)

In [77]:
#simple_fc = ClassicalComponent(num_classes, input_size)
train_model(hqcnn, training_loader, num_epochs, optimizer, loss_fn)

Passing through quanvolution layer...
Quanvolution processing time: 2.356858999992255
Epoch [1/4], Step [1/4], Loss: 0.7029, Acc: 0.4800
Passing through quanvolution layer...
Quanvolution processing time: 2.103883000003407
Epoch [1/4], Step [2/4], Loss: 0.7010, Acc: 0.4850
Passing through quanvolution layer...
Quanvolution processing time: 2.0533047999924747
Epoch [1/4], Step [3/4], Loss: 0.6972, Acc: 0.5000
Passing through quanvolution layer...
Quanvolution processing time: 2.0809785000019474
Epoch [1/4], Step [4/4], Loss: 0.6976, Acc: 0.5000
Epoch 1: Train Loss=0.6976, Train Acc=0.5000
Passing through quanvolution layer...
Quanvolution processing time: 2.122514400005457
Epoch [2/4], Step [1/4], Loss: 0.7043, Acc: 0.4600
Passing through quanvolution layer...
Quanvolution processing time: 2.2003954000101658
Epoch [2/4], Step [2/4], Loss: 0.6977, Acc: 0.4900
Passing through quanvolution layer...
Quanvolution processing time: 2.687404600001173
Epoch [2/4], Step [3/4], Loss: 0.6979, Acc: 

In [71]:
def train_model(model, train_loader, num_epochs, optimizer, loss_fn):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        epoch_loss = 0.0
        total = 0
        
        
        for i, (inputs, labels) in enumerate(train_loader):
        
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)  # Calculate loss
            
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            loss.backward()  # Backpropagate to calculate gradients
            optimizer.step() # Update weights
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/(i+1):.4f}, Acc: {correct / total:.4f}")
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss={epoch_loss:.4f}, Train Acc={epoch_acc:.4f}")
            # Print every 10 batches
            

In [53]:
torch.cuda.is_available()

False

In [54]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy().reshape(28, 28)
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

ValueError: cannot reshape array of size 200 into shape (28,28)

### Testing tensor dimensions and quantum convolution

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 4
dummy_input = torch.zeros(input_size, input_size, 2).numpy()
patched_images = [image.extract_patches_2d(img, (patch_size, patch_size)) for img in dummy_input]
#dummy_output = quanv.forward(dummy_input)

tensor(patched_images).shape

torch.Size([4, 3, 2, 2])

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 28

shape = (input_size, input_size)
dummy_input = np.zeros(shape, dtype=float)
patched_images = image.extract_patches_2d(dummy_input, (patch_size, patch_size)) 
#dummy_output = quanv.forward(dummy_input)
n_patches, patch_h, patch_w = tensor(patched_images).shape
patched_images = tensor(patched_images).contiguous().view(n_patches, patch_h*patch_w)
image._extract_patches()

In [ ]:
patched_images[0]

tensor([0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
patched_images.shape

torch.Size([729, 4])

In [ ]:
myAnsatz = AnsatzSimulation(n_qubits)
outputs = [myAnsatz.simulate_circuit(patch, 'rx', toy_chromosome) for patch in patched_images]

In [ ]:
tensor(outputs).shape

torch.Size([1458, 4])